**Franco Ortiz** :Data scientist

UFO-reports-analysis
# Problem Statement
### Questions to answer
- What time is more frequent to see them?
- How long did the experience take?
- What shapes are most common and where?
- Top words that described that moment (wordcloud)
- heat map with quantities and cities for number of sightings

# Data importing
#### modules

In [20]:
import pandas as pd
import datetime as dt
from dataprep.clean import clean_date
import numpy as np
import seaborn as sns

# set seed for reproducibility
np.random.seed(0)

In [4]:
df = pd.read_csv("data/ufo.csv")
df.head(20)

,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,00:00,Dieppe (Moncton) (Canada),NB,Canada,Oval,NaN,((HOAX??)) we wher looking .. in the sky in di...,9/24/12,NaN
1,4/ 23:22,Atlantic City,NJ,USA,Disk,30 minutes,Observed a very bright white saucer (had two l...,5/6/17,NaN
2,03142020 23:04,Norwich,VT,USA,Circle,97 seconds,Strange sighting in the sky,5/15/20,NaN
3,1970s 23:00,Grande Summit,PA,USA,Light,5 minutes,4 lights speeding very fast across sky silently,8/16/21,NaN
4,23:00,Desolation Wilderness,CA,USA,Triangle,2 min.,triangle of lights moving slowly through the n...,9/15/05,NaN
5,23:00,Playa del Rey,CA,USA,Fireball,3-5 seconds,"Neon green, long-tailed fireball meteor-lookin...",7/8/04,NaN
6,23:00,Milltown (Canada),NB,Canada,Circle,NaN,"I wonder where they are today, and if they are...",6/4/04,NaN
7,23:00,Manning,SC,USA,Sphere,2 minutes,The object was metallic in color. Spherical in...,6/25/20,NaN
8,23:00,New Alexandria,PA,USA,Other,15 minutes,Translucent UFO.,5/9/19,NaN
9,23:00,"Northern California Coast (in, highway)",CA,USA,Cigar,3-5 min.,We were travelling along the coast of Californ...,10/2/99,NaN


- As we see index 1,2,3,4 must be put in an standard format

In [3]:
df.columns

Index(['Date / Time', 'City', 'State', 'Country', 'Shape', 'Duration',
       'Summary', 'Posted', 'Images'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138298 entries, 0 to 138297
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date / Time  138197 non-null  object
 1   City         137909 non-null  object
 2   State        128886 non-null  object
 3   Country      138075 non-null  object
 4   Shape        132096 non-null  object
 5   Duration     131537 non-null  object
 6   Summary      138227 non-null  object
 7   Posted       138296 non-null  object
 8   Images       524 non-null     object
dtypes: object(9)
memory usage: 9.5+ MB


# Data Cleaning

#### splitting datetime column

In [5]:
# print the first few rows of the date column
print(df['Date / Time'].head())

0             00:00
1          4/ 23:22
2    03142020 23:04
3       1970s 23:00
4             23:00
Name: Date / Time, dtype: object


In [8]:
# check the data type of our date column
print(df['Date / Time'].dtype)

object


# EDA

#### what shapes are most common and where 

In [38]:
shapes_where = df.groupby(['City'])['Shape'].value_counts().sort_values(ascending=False).head(15)
shapes_where

City          Shape 
Seattle       Light     146
Phoenix       Light     144
Las Vegas     Light     131
Portland      Light     129
San Diego     Light     117
Tucson        Light      98
Los Angeles   Light      93
Myrtle Beach  Light      90
Springfield   Light      73
Orlando       Light      73
Miami         Light      72
Albuquerque   Light      71
Portland      Circle     70
Houston       Light      69
Las Vegas     Circle     68
Name: Shape, dtype: int64

# Feature Engineering

# Modelling

# Communication